In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]

load gdsc2


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [6]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for target_index in tqdm(np.arange(res.shape[dim])):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = nihgcn_new(
                cell_exprs=exprs,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/240 [00:00<?, ?it/s]

epoch:   0 loss:0.709002 auc:0.4520
epoch:  20 loss:0.153209 auc:0.7782
epoch:  40 loss:0.136905 auc:0.7480
epoch:  60 loss:0.123181 auc:0.7450
epoch:  80 loss:0.114529 auc:0.7411
epoch: 100 loss:0.108614 auc:0.7862
epoch: 120 loss:0.102904 auc:0.8392
epoch: 140 loss:0.097554 auc:0.8728
epoch: 160 loss:0.093757 auc:0.8831
epoch: 180 loss:0.089240 auc:0.8947
epoch: 200 loss:0.088499 auc:0.8843
epoch: 220 loss:0.083874 auc:0.9143
epoch: 240 loss:0.081974 auc:0.9151
epoch: 260 loss:0.079845 auc:0.9091
epoch: 280 loss:0.079814 auc:0.9015
epoch: 300 loss:0.077965 auc:0.9287
epoch: 320 loss:0.076641 auc:0.9257
epoch: 340 loss:0.076715 auc:0.9249
epoch: 360 loss:0.076369 auc:0.9216
epoch: 380 loss:0.075000 auc:0.9233
epoch: 400 loss:0.074535 auc:0.9273
epoch: 420 loss:0.074820 auc:0.9299
epoch: 440 loss:0.073995 auc:0.9334
epoch: 460 loss:0.074004 auc:0.9199
epoch: 480 loss:0.073494 auc:0.9366
epoch: 500 loss:0.074390 auc:0.9322
epoch: 520 loss:0.074599 auc:0.9098
epoch: 540 loss:0.073108 auc

  2%|▏         | 4/240 [00:26<26:08,  6.64s/it]

Fit finished.
epoch:   0 loss:0.698697 auc:0.4932
epoch:  20 loss:0.153377 auc:0.7558
epoch:  40 loss:0.137354 auc:0.8113
epoch:  60 loss:0.123877 auc:0.8148
epoch:  80 loss:0.115146 auc:0.8062
epoch: 100 loss:0.109181 auc:0.7927
epoch: 120 loss:0.103416 auc:0.7716
epoch: 140 loss:0.098962 auc:0.7643
epoch: 160 loss:0.093636 auc:0.7479
epoch: 180 loss:0.091060 auc:0.6975
epoch: 200 loss:0.086107 auc:0.7798
epoch: 220 loss:0.084773 auc:0.8030
epoch: 240 loss:0.081823 auc:0.7354
epoch: 260 loss:0.080202 auc:0.7794
epoch: 280 loss:0.078778 auc:0.7577
epoch: 300 loss:0.077666 auc:0.7669
epoch: 320 loss:0.077196 auc:0.7720
epoch: 340 loss:0.076178 auc:0.7720
epoch: 360 loss:0.075723 auc:0.7613
epoch: 380 loss:0.075019 auc:0.7896
epoch: 400 loss:0.074900 auc:0.7876
epoch: 420 loss:0.074671 auc:0.8020
epoch: 440 loss:0.074495 auc:0.7952
epoch: 460 loss:0.076313 auc:0.8188
epoch: 480 loss:0.073850 auc:0.7855
epoch: 500 loss:0.073621 auc:0.7843
epoch: 520 loss:0.075731 auc:0.7240
epoch: 540 los

  5%|▍         | 11/240 [00:43<13:55,  3.65s/it]

...

epoch: 980 loss:0.072214 auc:0.7399
Fit finished.
epoch:   0 loss:0.696002 auc:0.5026
epoch:  20 loss:0.153447 auc:0.8319
epoch:  40 loss:0.137295 auc:0.7452
epoch:  60 loss:0.123405 auc:0.6070
epoch:  80 loss:0.114458 auc:0.5783
epoch: 100 loss:0.108438 auc:0.6307
epoch: 120 loss:0.102568 auc:0.6898
epoch: 140 loss:0.097056 auc:0.7445
epoch: 160 loss:0.093044 auc:0.7599
epoch: 180 loss:0.088507 auc:0.7870
epoch: 200 loss:0.085196 auc:0.8018
epoch: 220 loss:0.084389 auc:0.8455
epoch: 240 loss:0.080963 auc:0.8199
epoch: 260 loss:0.081760 auc:0.8483
epoch: 280 loss:0.078529 auc:0.8593
epoch: 300 loss:0.077357 auc:0.8587
epoch: 320 loss:0.077130 auc:0.8707
epoch: 340 loss:0.076168 auc:0.8553
epoch: 360 loss:0.075766 auc:0.8661
epoch: 380 loss:0.076778 auc:0.8997
epoch: 400 loss:0.074920 auc:0.8600
epoch: 420 loss:0.076273 auc:0.8353
epoch: 440 loss:0.074325 auc:0.8707
epoch: 460 loss:0.074276 auc:0.8781
epoch: 480 loss:0.073776 auc:0.8618
epoch: 500 loss:0.075387 auc:0.8717
epoch: 520 los

 98%|█████████▊| 235/240 [30:27<00:23,  4.70s/it]

epoch: 980 loss:0.072189 auc:0.8467
Fit finished.
epoch:   0 loss:0.709369 auc:0.4578
epoch:  20 loss:0.154311 auc:0.6470
epoch:  40 loss:0.138441 auc:0.6841
epoch:  60 loss:0.124370 auc:0.7667
epoch:  80 loss:0.115125 auc:0.7065
epoch: 100 loss:0.108947 auc:0.6662
epoch: 120 loss:0.103008 auc:0.6823
epoch: 140 loss:0.097360 auc:0.7352
epoch: 160 loss:0.092969 auc:0.7700
epoch: 180 loss:0.088841 auc:0.7647
epoch: 200 loss:0.085902 auc:0.7848
epoch: 220 loss:0.083991 auc:0.7798
epoch: 240 loss:0.083346 auc:0.8196
epoch: 260 loss:0.080092 auc:0.7727
epoch: 280 loss:0.079275 auc:0.7906
epoch: 300 loss:0.077962 auc:0.8289
epoch: 320 loss:0.077089 auc:0.8161
epoch: 340 loss:0.076182 auc:0.8196
epoch: 360 loss:0.075554 auc:0.8216
epoch: 380 loss:0.076171 auc:0.8534
epoch: 400 loss:0.074933 auc:0.8486
epoch: 420 loss:0.074424 auc:0.8468
epoch: 440 loss:0.075910 auc:0.8468
epoch: 460 loss:0.074117 auc:0.8602
epoch: 480 loss:0.073703 auc:0.8566
epoch: 500 loss:0.073471 auc:0.8554
epoch: 520 los

100%|██████████| 240/240 [30:37<00:00,  7.66s/it]

epoch: 980 loss:0.072595 auc:0.8733
Fit finished.


In [7]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")

In [8]:
drug_sum

array([  8.,   7.,   0., 160.,   0.,   0.,   2.,   2.,   1.,   1., 115.,
         0.,   5.,  42.,  17.,  22., 154.,  32.,  16.,  91., 104.,   4.,
        54., 431., 394.,  19.,   5.,  43.,  83.,  44., 201.,   4.,   0.,
        34.,  24., 573.,   2.,   2.,  56., 123.,   2.,   0.,  35.,  34.,
       894.,  39.,  26.,  74.,   0.,   0.,   0.,   6.,   0., 778.,  12.,
        19.,  30.,   0., 176.,  27.,   0., 675., 877., 768., 467., 244.,
       180., 779., 898., 464.,   1.,   0.,   0.,   0.,   0., 145.,  79.,
         2., 653.,  22.,  15., 187.,   1.,   0.,   0., 296.,   4.,   0.,
        20.,   0.,   3.,  17.,   0.,   0.,   4.,   3.,   1.,  15., 551.,
        26.,   0.,  12.,   0.,   0.,   1.,  26.,  20.,  78.,  12.,   0.,
       147.,   0.,   0.,   5.,   0.,   0., 425.,   0.,  18.,   0.,   0.,
       445.,   4., 774., 876.,   0.,   0., 341.,  41.,  33.,   0.,   0.,
       364.,   0., 273.,   3.,  83.,   0.,   2.,  16., 178.,   0.,  16.,
         3.,   0.,  26.,  99., 250.,   0.,  78.,  1

In [9]:
cell_sum

array([32., 35., 36., 21., 29., 31., 34., 73., 36., 38., 38., 25., 24.,
       37., 31., 33., 37., 28., 59., 66., 45., 29., 68., 38., 31., 22.,
       47., 43., 19., 24., 42., 50., 56., 52., 35., 56., 29., 58., 29.,
       61., 36.,  3., 54., 40., 37., 19., 58., 56., 34., 19., 48., 35.,
       33., 15., 26., 34., 23., 21., 48., 48., 20., 16.,  3., 24., 53.,
       29., 23., 42., 29., 19., 34., 16., 31., 41., 22., 21., 21., 36.,
       40., 38., 36., 54., 39., 46., 38., 32., 27., 28., 27., 24., 22.,
       35., 52., 45., 26., 20., 30., 32., 29., 55., 24., 15., 21., 25.,
       64., 43., 42., 32.,  8., 36., 12., 32., 25., 31., 30., 26., 15.,
       27., 27., 28., 30., 37., 27., 23., 12.,  8., 31., 21., 33., 39.,
       31., 43., 59., 24., 33., 37., 33., 23., 31., 24., 14., 21., 24.,
       54., 27., 21., 27., 17., 16., 24., 30., 44., 13., 64., 42., 23.,
       21., 30., 40., 22., 29., 40., 67., 47., 37., 41., 23., 55., 36.,
       38., 42., 30., 37., 42., 35., 43., 37., 24., 26., 17., 12